In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import random
import os
from tensorflow.python.framework import ops
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.utils import to_categorical, normalize 
import h5py


In [3]:
train_dir="/media/dheeraj/9A26F0CB26F0AA01/WORK/jupyter_files/PlantDiseaseDetection/train/train"

In [4]:
test_dir="/media/dheeraj/9A26F0CB26F0AA01/WORK/jupyter_files/PlantDiseaseDetection/testpicture"

In [5]:
def label_img_chutiya(img):
    word_label = img[0]
  
    if word_label == 'h': return 0
    
    elif word_label == 'b': return 1
    elif word_label == 'v': return 2
    elif word_label == 'l': return 3

In [6]:
#print(os.listdir(train_dir))
for img in os.listdir(train_dir):
    path = os.path.join(train_dir,img)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    #img_array=cv2.imread(img)  #converting the photo to greyscale
    #plt.imshow(img)
    #plt.show()
    #break
    #break    
    

In [ ]:
#print(img)

In [7]:
#The sizes of all the photos are different. 
# changing the size of every photo to a constant size to work for CNN

img_size= 100

new_images=cv2.resize(img, (img_size, img_size))  # size of image is 100*100
#plt.imshow(new_images)
#plt.show()


In [8]:
training_data=[]
def create_training_data():
    for img in os.listdir(train_dir):
        #try:
        label = label_img_chutiya(img)
        path = os.path.join(train_dir,img)
        img=cv2.imread(path, cv2.IMREAD_COLOR)  
        new_images=cv2.resize(img, (img_size, img_size))  # size of image is 100*100
        training_data.append([new_images, label])
        #print(class_num)
        #except Exception as e:
         #   pass

create_training_data()      
#print(training_data[0])

In [9]:
testing_data = []
def create_test_data():
    
    for img in (os.listdir(test_dir)):
        label = label_image(img)
        path = os.path.join(TEST_DIR, img)
        #img_num = img.split('.')[0]
        
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), np.array(label)])
    #np.save("testing_data.npy", testing_data)
    return testing_data

In [10]:
print(len(training_data))

4000


In [11]:
random.shuffle(training_data)

In [12]:
x=[]
y=[]

for features, labels in training_data:
    x.append(features)
    y.append(labels)
#plt.imshow(x[0])


#converting the pixels into a numpy array
#-1 for including all the images
# img_size for the image size that we have taken
# 1 since the image has been taken in a grayscale format
x=np.array(x).reshape([-1, img_size, img_size, 3])
print(x.shape)
y=to_categorical(y,4)
y=np.array(y)
print(y[1000])           

(4000, 100, 100, 3)
[0. 0. 1. 0.]


In [13]:
#normalising the data by dividing it by the maximum value of pixel

x=x/255.0


In [14]:

#Since the model is Sequential
model=Sequential()


#First Convolutional Layer

#convolving the input layer with a 3*3 filter 
model.add(Conv2D(64, (3,3), input_shape=x.shape[1:]))

#after Convolution, we can either pass activation function of Maxpooling
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Second Convoutional Layer

model.add(Conv2D(64, (3,3)))

model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


#Adding the fully connected Layer

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

#Adding the output Layer
model.add(Dense(4))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=['accuracy'])


model.fit(x, y, batch_size=32,epochs=3, validation_split=0.1)
y1=model.predict(x)


Train on 3600 samples, validate on 400 samples
Epoch 1/3
3600/3600 [==============================] - 141s 39ms/step - loss: 0.6975 - acc: 0.7014 - val_loss: 0.2801 - val_acc: 0.8950
Epoch 2/3
3600/3600 [==============================] - 145s 40ms/step - loss: 0.2680 - acc: 0.9047 - val_loss: 0.3042 - val_acc: 0.8800
Epoch 3/3
3600/3600 [==============================] - 152s 42ms/step - loss: 0.1627 - acc: 0.9431 - val_loss: 0.1852 - val_acc: 0.9200


In [15]:
print(y1)
print(y1.shape)
#print(to_categorical(y1,4))

[[1.0567595e-04 8.0498832e-01 1.8823144e-01 6.6745868e-03]
 [9.8149008e-01 1.5598596e-02 8.7642245e-04 2.0348732e-03]
 [2.0633517e-03 9.9040133e-01 3.6859405e-03 3.8494426e-03]
 ...
 [2.5906721e-03 9.9120343e-01 3.9216182e-03 2.2842397e-03]
 [2.2565104e-02 9.7060150e-01 4.8249471e-03 2.0084390e-03]
 [8.0198003e-04 4.5070833e-01 1.6689773e-01 3.8159195e-01]]
(4000, 4)


In [16]:
keras_file = "keras_model.h5"
tf.keras.models.save_model(model, keras_file)

# Convert to TensorFlow Lite model.
#converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

/home/dheeraj/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


8825424

In [ ]:
'''output_names = [node.op.name for node in model.outputs]
sess = tf.keras.backend.get_session()
frozen_def = tf.graph_util.convert_variables_to_constants(
    sess, sess.graph_def, output_names)'''

In [17]:
import sys

def apply_mask(matrix, mask, fill_value):

    #print(flat[60])
    #print(flat[11940])
        
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    #print('MASKED=',masked)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)
    #print('Low MASK->',low_mask,'\nMatrix->',matrix)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100

    half_percent = percent / 200.0
    #print('HALF PERCENT->',half_percent)

    channels = cv2.split(img)
    #print('Channels->\n',channels)
    #print('Shape->',channels[0].shape)
    #print('Shape of channels->',len(channels[2]))

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)
        #print('vec=',vec_size,'\nFlat=',flat)
        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        #print ("Lowval: ", low_val)
        #print ("Highval: ", high_val)

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)



In [18]:
import os
import sys
import cv2
import numpy
files = os.listdir(test_dir)
filepaths = [os.path.join(test_dir,i) for i in files]
#print(filepaths)
print("Directory= \n\n\n\n", test_dir.split('/'))
for i in filepaths:
    img = cv2.imread(i)
    try:
        img.shape[2] == 3
    except:
        continue
    out = simplest_cb(img, 1)
    cv2.imshow("Before", img)
    cv2.imshow("After", out)
    #cv2.waitKey(0)
    cv2.waitKey(0)
    #cv2.destoyAllwindows()
        
    
    out=np.array(out)
    out=cv2.resize(out, (img_size,img_size))
    out=np.array(out).reshape([-1, img_size, img_size, 3])
    y1=model.predict(out)
    #print(y1)

    for i in y1:
        if i[0]==1:
            print("Thecurrent status of the crop is: Healthy \n\nNo further action needed.")
        elif(i[1]==3):
            print("The crop is suffering from :Lisianthus\n\nRemedy:Monitor the field, handpick diseased plants and bury them. \n  Use sticky yellow plastic traps. \n  Spray insecticides such as organophosphates, carbametes during the seedliing stage. \n Use copper fungicites")
        elif(i[2]==1):
            print("The crop is suffering from :Bromaliaceae\n\nRemedy:Discard or destroy any affected plants. \n  Do not compost them. \n  Rotate yoour tomato plants yearly to prevent re-infection next year. \n Use copper fungicites")
        else:
            print("The crop is suffering from :Vitaceae\n\nDiscard or destroy any affected plants. \n  Do not compost them. \n  Rotate yoour tomato plants yearly to prevent re-infection next year. \n Use copper fungicites")

    


Directory= 



 ['', 'media', 'dheeraj', '9A26F0CB26F0AA01', 'WORK', 'jupyter_files', 'PlantDiseaseDetection', 'testpicture']


NotFoundError: PruneForTargets: Some target nodes not found: group_deps_1 